## LINKTELLER: About the paper

In this paper, we focus on the edge privacy, and consider a training scenario here the data holder Bob with node features will first send training node features to Alice who owns the adjacency information. Alice will then train
a graph neural network (GNN) with the joint information and provide an inference API to Bob. During inference time, Bob is able to provide test node features and query the API to obtain the predictions for test nodes. Under this setting, we first propose a privacy attack LINKTELLER via influence analysis to infer the private edge information held by Alice via designing adversarial queries for Bob.

## Libraries

In [1]:
!pip -q install --index-url https://download.pytorch.org/whl/cu121 torch==2.3.1

# 2) PyG and companions compiled for torch 2.3.1 + cu121
!pip -q install -f https://data.pyg.org/whl/torch-2.3.1+cu121.html \
  torch_geometric==2.5.3 torch_scatter==2.1.2 torch_sparse==0.6.18

# 3) Pin fsspec to avoid the LocalFileSystem.mv() signature mismatch
!pip -q install --force-reinstall --no-deps fsspec==2023.6.0

# (Optional utilities)
!pip -q install scikit-learn networkx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 111.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.6 MB/s eta 0:00:00
ERROR: pip's dependency resol

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import to_networkx, dense_to_sparse
from torch_geometric.nn import GCNConv
import networkx as nx
from sklearn.cluster import KMeans
import torch, fsspec, torch_geometric
from torch_geometric.datasets import TUDataset
import numpy as np
from sklearn.model_selection import train_test_split
import math, random
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cpu')

## Loading Dataset

Also preprocess labels

In [3]:
from torch_geometric.transforms import BaseTransform
class graphToNodeLabel(BaseTransform):
    def __call__(self, data):
        # Ensure x (node features) exists and is a tensor
        y = data.y
        node_size = data.x.size(0)
        y_expanded = y.expand(node_size)
        data.y = y_expanded
        return data


In [4]:
dataset = TUDataset(root='data/TUD', name='MUTAG', transform=graphToNodeLabel())  # 188 graphs
sizes = [data.num_nodes for data in dataset]
idx = int(np.argmax([n if n >= 25 else 0 for n in sizes]))  # pick a larger graph
data = dataset[idx]
print(data)
#print(f"Graph index {idx}: nodes={data.num_nodes}, edges={data.num_edges // 2} (undirected)")


Data(edge_index=[2, 62], x=[28, 7], edge_attr=[62, 4], y=[28])


Processing...
Done!


# Aggregate Dataset into 1 Batch for a large single graph

Linkteller makes no assumption of a single connected component, so we can set the node features to the graph classification and aggregate all graphs into a single graph

In [5]:
from torch_geometric.data import Batch

print(dataset[0])
#aggregate all MUTAG data into one graph
big_graph = Batch.from_data_list([data for data in dataset])
print(big_graph)
data = big_graph

Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[17])
DataBatch(edge_index=[2, 7442], x=[3371, 7], edge_attr=[7442, 4], y=[3371], batch=[3371], ptr=[189])


## BUild X features

In [6]:
X = data.x.float()  # [N, D]
N, D = X.shape


#K = min(3, len(torch.unique(X, dim=0))) if len(torch.unique(X, dim=0))>1 else 2
#km = KMeans(n_clusters=K, n_init=10, random_state=0).fit(X.numpy())
#y_node = torch.from_numpy(km.labels_).long()

# Train/val/test node splits
idx_all = np.arange(N)
idx_train, idx_tmp = train_test_split(idx_all, test_size=0.4, random_state=42, stratify=data.y.numpy())
idx_val, idx_test = train_test_split(idx_tmp, test_size=0.5, random_state=42)

train_mask = torch.zeros(N, dtype=torch.bool); train_mask[idx_train] = True
val_mask   = torch.zeros(N, dtype=torch.bool); val_mask[idx_val] = True
test_mask  = torch.zeros(N, dtype=torch.bool); test_mask[idx_test] = True

print(f"Splits: train {train_mask.sum().item()}, val {val_mask.sum().item()}, test {test_mask.sum().item()}")


Splits: train 2022, val 674, test 675


## Adjacency (A) helpers

In [7]:
# Edge index is undirected in PyG; keep it as-is
edge_index = data.edge_index  # [2, E]

# For evaluation convenience, build a boolean adjacency (without self loops)
A = torch.zeros((N, N), dtype=torch.bool)
A[edge_index[0], edge_index[1]] = True
A[edge_index[1], edge_index[0]] = True
A.fill_diagonal_(False)
true_edges_undirected = torch.nonzero(torch.triu(A, diagonal=1), as_tuple=False)  # [M, 2]
M_true = true_edges_undirected.shape[0]
density = M_true / (N*(N-1)/2)
print(f"True undirected edges: {M_true} | density={density:.4f}")


True undirected edges: 3721 | density=0.0007


## 3 small layer GCN for node classification

In [ ]:
class EdgeMLP(nn.Module):
    """Small MLP to combine node and edge features inside each GINE layer."""
    def __init__(self, input_dim, hidden_dim, dropout=0.0):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [149]:
class GCN(nn.Module):
    def __init__(self, in_channels, hidden, out_channels, dropout=0.2,
                 p_noise = 0.2, num_node_feats = 7, node_embed_dim = 7):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden)
        self.conv2 = GCNConv(hidden, hidden)
        self.conv3 = GCNConv(hidden, out_channels)
        self.dropout = dropout
        self.p_noise = p_noise

    def add_noisy_edges(self, edge_index, num_nodes):
        # Sample random edges
        num_rand = int(self.p_noise * edge_index.size(1))

        # uniformly pick random node pairs
        row = torch.randint(0, num_nodes, (num_rand,))
        col = torch.randint(0, num_nodes, (num_rand,))
        noisy_edges = torch.stack([row, col], dim=0)

        # concat & avoid duplicates (optional)
        edge_index_noisy = torch.cat([edge_index, noisy_edges], dim=1)
        return edge_index_noisy

    def forward(self, x, edge_index):
        if self.training:
          edge_index = self.add_noisy_edges(edge_index, x.size(0))
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv3(x, edge_index)
        x = F.sigmoid(x) #for binary cross entropy
        return x  # logits (N, K)
#binary classification, so 1 channel
model = GCN(D, hidden=32, out_channels=1, dropout=0.1).to(device)
X_dev = X.to(device)
edge_index_dev = edge_index.to(device)
y_dev = data.y.to(device)
y_dev = torch.unsqueeze(y_dev, 1).float() #add dimension
train_mask_dev = train_mask.to(device)
val_mask_dev   = val_mask.to(device)
test_mask_dev  = test_mask.to(device)
print(train_mask_dev)
print(y_dev[train_mask_dev])


tensor([ True, False, False,  ..., False,  True,  True])
tensor([[1.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]])


## train the model

In [150]:
def train_model(model, X_dev, y_dev, edge_index_dev,
                train_mask_dev, val_mask_dev):
    opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    best_val, best_state = -1, None

    for epoch in range(400):
        model.train()
        opt.zero_grad()
        logits = model(X_dev, edge_index_dev)
        loss = F.binary_cross_entropy(logits[train_mask_dev], y_dev[train_mask_dev])
        loss.backward()
        opt.step()
        if (epoch + 1) % 50 == 0: #train loss
          print(f"Train loss for epoch {epoch + 1}: {loss:.3f}")

        # quick val acc
        model.eval()
        with torch.no_grad():
            val_pred = torch.round(logits[val_mask_dev])
            val_acc = (val_pred == y_dev[val_mask_dev]).float().mean().item()
            if (epoch + 1) % 50 == 0:
              print(f"Val ACC for epoch {epoch + 1}: {val_acc:.3f}")
        if val_acc > best_val:
            best_val = val_acc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

    print(f"Best val acc: {best_val:.3f}")
    model.load_state_dict({k: v for k, v in best_state.items()})


In [151]:
train_model(model, X_dev, y_dev, edge_index_dev, train_mask_dev, val_mask_dev)

Train loss for epoch 50: 0.553
Val ACC for epoch 50: 0.723
Train loss for epoch 100: 0.526
Val ACC for epoch 100: 0.748
Train loss for epoch 150: 0.528
Val ACC for epoch 150: 0.751
Train loss for epoch 200: 0.531
Val ACC for epoch 200: 0.754
Train loss for epoch 250: 0.531
Val ACC for epoch 250: 0.737
Train loss for epoch 300: 0.524
Val ACC for epoch 300: 0.752
Train loss for epoch 350: 0.518
Val ACC for epoch 350: 0.743
Train loss for epoch 400: 0.530
Val ACC for epoch 400: 0.757
Best val acc: 0.779


## Black-box “API” wrapper (returns logits for chosen nodes)

In [79]:
@torch.no_grad()
def gbb_api(model, node_ids, X_query):
    """
    node_ids: 1D LongTensor of node indices to fetch from output
    X_query: (N, D) full feature matrix Bob provides (Alice uses it with her private edge_index)
    returns: logits[node_ids] shape (len(node_ids), K)
    """
    model.eval()
    out = model(X_query.to(device), edge_index_dev)  # full-graph forward
    return out[node_ids.to(device)].detach().cpu()


## LINKTELLER influence matrix & scoring

In [82]:
def influence_matrix_for_v(model, v, V_I, X_base, delta=1e-2):
    """
    v: node index (int)
    V_I: 1D LongTensor of nodes-of-interest to score against
    X_base: (N, D) baseline features
    returns: Iv (|V_I|, K) = (P' - P)/delta where rows correspond to u in V_I
    """
    node_ids = V_I
    P = gbb_api(model, node_ids, X_base)

    Xp = X_base.clone()
    Xp[v] = (1.0 + delta) * Xp[v]  # upweight features of v
    Pp = gbb_api(model, node_ids, Xp)

    Iv = (Pp - P) / delta  # finite-diff approximation
    return Iv  # (|V_I|, K)

def linkteller_scores(model, V_C, X_base, delta=1e-2):
    """
    V_C: nodes-of-interest (attack surface) as 1D LongTensor
    returns: dict {(u,v): score} for u != v, unordered pairs
    """
    V_C = V_C.cpu()
    scores = {}
    for j, v in enumerate(V_C.tolist()):
        Iv = influence_matrix_for_v(model, v, V_C, X_base, delta=delta).numpy()  # rows aligned with V_C
        # influence value of v on each u = ||Iv[u,:]||_2
        norms = np.linalg.norm(Iv, axis=1)
        for i, u in enumerate(V_C.tolist()):
            if u == v:
                continue
            key = (min(u,v), max(u,v))
            # symmetrical score: max of v→u and u→v will be handled later; accumulate max
            scores[key] = max(scores.get(key, 0.0), float(norms[i]))
    return scores

# Choose attack node set V_C (we’ll use all nodes to make life easy)
V_C = torch.arange(N, dtype=torch.long)
scores = linkteller_scores(model, V_C, X, delta=1e-2)

# Turn scores into a sorted list
sorted_pairs = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
len(sorted_pairs), sorted_pairs[:5]


(5680135,
 [((2397, 2398), 0.40415525436401367),
  ((2792, 2793), 0.3863096237182617),
  ((2049, 2050), 0.3590106964111328),
  ((1132, 1133), 0.3544062376022339),
  ((2679, 2680), 0.3474622964859009)])

## Pick top-m pairs using a density belief k̂

In [84]:
n = N
m_true = M_true
m_belief = int(round(density * (n*(n-1)/2)))

pred_edges = set([pair for (pair, _) in sorted_pairs[:m_belief]])

# ground truth undirected edges as set of tuples (i,j) with i<j
true_edges = set([tuple(e.tolist()) for e in true_edges_undirected])

tp = len(pred_edges & true_edges)
fp = len(pred_edges - true_edges)
fn = len(true_edges - pred_edges)

precision = tp / (tp + fp + 1e-12)
recall    = tp / (tp + fn + 1e-12)
f1        = 2*precision*recall / (precision + recall + 1e-12)
print(f"Precision={precision:.3f} | Recall={recall:.3f} | F1={f1:.3f} | m_belief={m_belief} | true M={m_true}")

Precision=0.829 | Recall=0.829 | F1=0.829 | m_belief=3721 | true M=3721


##Sweep density belief k̂ to see sensitivity

In [50]:
def evaluate_at_fraction(frac):
    m = int(round(frac * (n*(n-1)/2)))
    pred = set([pair for (pair, _) in sorted_pairs[:m]])
    tp = len(pred & true_edges)
    fp = len(pred - true_edges)
    fn = len(true_edges - pred)
    p = tp / (tp + fp + 1e-12)
    r = tp / (tp + fn + 1e-12)
    f1 = 2*p*r / (p + r + 1e-12)
    return p, r, f1, m

for frac in [0.5*density, 0.8*density, density, 1.2*density, 1.5*density]:
    p, r, f1, m = evaluate_at_fraction(frac)
    print(f"k_hat={frac:.4f}  m={m:3d}  P={p:.3f} R={r:.3f} F1={f1:.3f}")


k_hat=0.0003  m=1861  P=0.896 R=0.448 F1=0.598
k_hat=0.0005  m=2977  P=0.867 R=0.694 F1=0.771
k_hat=0.0007  m=3721  P=0.808 R=0.808 F1=0.808
k_hat=0.0008  m=4465  P=0.736 R=0.883 F1=0.803
k_hat=0.0010  m=5582  P=0.613 R=0.919 F1=0.735


Calculate AUROC

In [83]:
from sklearn.metrics import roc_auc_score
def calculate_auc(scores: dict[tuple:float], true_edges: set[tuple]):
  #add set of false edges (every edge in score not in true_edges)
  false_edges = set(scores.keys()).difference(true_edges)
  #create prediction, score vectors in correct order
  y_true = [1] * len(true_edges) + [0] * len(false_edges)
  y_score = [scores[e] for e in true_edges] + [scores[e] for e in false_edges]
  return roc_auc_score(y_true, y_score)


auc = calculate_auc(scores, true_edges)
print("AUROC:", auc)



KeyboardInterrupt: 

# Dropout + Dummy Edge Test

In [152]:
def dropOutDummyEdgeTest(X_dev, y_dev, edge_index_dev,
                train_mask_dev, val_mask_dev, dropoutProb, p_noise):
    """
    Testing to get model attack success for different dropout/dummy edge
    incorporation rates
    """
    model = GCN(D, hidden=64, out_channels=1, dropout=dropoutProb, p_noise = p_noise).to(device)
    train_model(model, X_dev, y_dev, edge_index_dev, train_mask_dev, val_mask_dev)
    # Choose attack node set V_C (we’ll use all nodes to make life easy)
    V_C = torch.arange(N, dtype=torch.long)
    scores = linkteller_scores(model, V_C, X, delta=1e-2)

    # Turn scores into a sorted list
    sorted_pairs = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
    len(sorted_pairs), sorted_pairs[:5]

    n = N
    m_true = M_true
    m_belief = int(round(density * (n*(n-1)/2)))

    pred_edges = set([pair for (pair, _) in sorted_pairs[:m_belief]])

    # ground truth undirected edges as set of tuples (i,j) with i<j
    true_edges = set([tuple(e.tolist()) for e in true_edges_undirected])

    tp = len(pred_edges & true_edges)
    fp = len(pred_edges - true_edges)
    fn = len(true_edges - pred_edges)

    precision = tp / (tp + fp + 1e-12)
    recall    = tp / (tp + fn + 1e-12)
    f1        = 2*precision*recall / (precision + recall + 1e-12)
    print(f"Precision={precision:.3f} | Recall={recall:.3f} | F1={f1:.3f} | m_belief={m_belief} | true M={m_true}")

In [154]:
for i in range(10):
  print(f"Dropout Rate: {i/10}")
  dropOutDummyEdgeTest(X_dev, y_dev, edge_index_dev,
                  train_mask_dev, val_mask_dev, i/10, 0)


Dropout Rate: 0.0
Train loss for epoch 50: 0.509
Val ACC for epoch 50: 0.766
Train loss for epoch 100: 0.505
Val ACC for epoch 100: 0.773
Train loss for epoch 150: 0.503
Val ACC for epoch 150: 0.773
Train loss for epoch 200: 0.501
Val ACC for epoch 200: 0.779
Train loss for epoch 250: 0.500
Val ACC for epoch 250: 0.782
Train loss for epoch 300: 0.499
Val ACC for epoch 300: 0.782
Train loss for epoch 350: 0.498
Val ACC for epoch 350: 0.782
Train loss for epoch 400: 0.496
Val ACC for epoch 400: 0.785
Best val acc: 0.785
Precision=0.732 | Recall=0.732 | F1=0.732 | m_belief=3721 | true M=3721
Dropout Rate: 0.1
Train loss for epoch 50: 0.514
Val ACC for epoch 50: 0.764
Train loss for epoch 100: 0.508
Val ACC for epoch 100: 0.769
Train loss for epoch 150: 0.502
Val ACC for epoch 150: 0.769
Train loss for epoch 200: 0.505
Val ACC for epoch 200: 0.776
Train loss for epoch 250: 0.504
Val ACC for epoch 250: 0.773
Train loss for epoch 300: 0.503
Val ACC for epoch 300: 0.773
Train loss for epoch 3

In [155]:
for i in range(11):
  print(f"Noisy Edge Prob.: {i/10}")
  dropOutDummyEdgeTest(X_dev, y_dev, edge_index_dev,
                  train_mask_dev, val_mask_dev, 0.5, i / 10)


Noisy Edge Prob.: 0.0
Train loss for epoch 50: 0.520
Val ACC for epoch 50: 0.745
Train loss for epoch 100: 0.507
Val ACC for epoch 100: 0.758
Train loss for epoch 150: 0.515
Val ACC for epoch 150: 0.757
Train loss for epoch 200: 0.514
Val ACC for epoch 200: 0.758
Train loss for epoch 250: 0.510
Val ACC for epoch 250: 0.761
Train loss for epoch 300: 0.513
Val ACC for epoch 300: 0.752
Train loss for epoch 350: 0.513
Val ACC for epoch 350: 0.763
Train loss for epoch 400: 0.510
Val ACC for epoch 400: 0.770
Best val acc: 0.774
Precision=0.794 | Recall=0.794 | F1=0.794 | m_belief=3721 | true M=3721
Noisy Edge Prob.: 0.1
Train loss for epoch 50: 0.534
Val ACC for epoch 50: 0.724
Train loss for epoch 100: 0.530
Val ACC for epoch 100: 0.754
Train loss for epoch 150: 0.533
Val ACC for epoch 150: 0.749
Train loss for epoch 200: 0.524
Val ACC for epoch 200: 0.748
Train loss for epoch 250: 0.522
Val ACC for epoch 250: 0.748
Train loss for epoch 300: 0.530
Val ACC for epoch 300: 0.752
Train loss for

In [143]:
model = GCN(D, hidden=32, out_channels=1, dropout=1).to(device)
y_pred = model(X_dev, edge_index_dev)
print(torch.sum(y_pred[val_mask_dev] <= 0.5)/y_pred[val_mask_dev].size(0))

tensor(1.)
